# Assignment 1: Data Parsing, Cleansing and Integration
## Task 3
#### Student Name: Nguyen Duc Quang
#### Student ID: 3927198

Date: 3/9/2023

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment, e.g.,:
* pandas
* numpy


## Introduction
This is the notebook describing the process of intergrating two datasets from different sources and different format. The datasets are job advertisements.


##  Import libraries 

In [89]:
# Code to import libraries as you need in this assessment
import pandas as pd
import numpy as np

### 1. Examining and loading data
Examine "\<student\_id\>_dataset2.csv" the structure and schema. 

In [90]:
df2 = pd.read_csv("./s3927198_dataset2.csv")
df2.head()

,Monthly Payment,Closing,Organisation,Location,Category,Full-Time Equivalent (FTE),Opening,Job Title,Type
0,1200.00,2012-12-27 12:00:00,Protocol Education,UK,Education,0.8,2012-11-27 12:00:00,Higher Level Teaching Assistant (HLTA),NaN
1,3125.00,2013-08-25 15:00:00,Cordius Ltd,UK,Information Technology,1.0,2013-06-26 15:00:00,Software Developers / Software Architects Not...,Permanent
2,4333.33,2013-01-16 15:00:00,Understanding Recruitment,Walton-On-Thames,Information Technology,1.0,2012-10-18 15:00:00,Senior BI Consultant (Cognos),Permanent
3,3750.00,2013-02-13 15:00:00,Wallace Hind Selection,UK,Sales,1.0,2012-12-15 15:00:00,"Technical Sales Manager, Pigments Fillers to S...",NaN
4,2916.67,2013-01-12 12:00:00,Greenhill Group Ltd,Eastern England,Sales,1.0,2012-12-29 12:00:00,Channel Account Manager Software,NaN


In [91]:
df1 = pd.read_csv('./s3927198_dataset1_solution.csv', index_col=0 )
df1.head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,71846213,"Nurse : Medical/Surgical Roles NHS Trust, SW E...",Devon,non-specified,non-specified,permanent,Healthcare & Nursing Jobs,24400.0,2012-02-03 15:00:00,2012-03-04 15:00:00,jobs4medical.co.uk
1,68580301,Dental : Associate Dentist Jobs in Somerset,Somerset,non-specified,non-specified,non-specified,Healthcare & Nursing Jobs,59000.0,2012-09-26 00:00:00,2012-11-25 00:00:00,jobs4medical.co.uk
2,71337916,Allied Health Care Professional : Optical Advi...,Cheshire,non-specified,non-specified,non-specified,Healthcare & Nursing Jobs,14000.0,2012-11-28 15:00:00,2013-01-27 15:00:00,jobs4medical.co.uk
3,71353660,Dental : Associates Jobs Dentist Job in Darlin...,Durham,non-specified,non-specified,non-specified,Healthcare & Nursing Jobs,28500.0,2013-05-24 00:00:00,2013-06-07 00:00:00,jobs4medical.co.uk
4,71337991,Allied Health Care Professional : Dispensing O...,Essex,non-specified,non-specified,non-specified,Healthcare & Nursing Jobs,22000.0,2013-07-11 00:00:00,2013-08-10 00:00:00,jobs4medical.co.uk


### 2. Resolving schema conflicts

#### 2.1 Schema conflict: Category columns values does not match for same categories

In [92]:
# Code to inspect data and identify conflicts in schema
category1 = df1.Category.value_counts().index.tolist()
category1

['IT Jobs',
 'Healthcare & Nursing Jobs',
 'Engineering Jobs',
 'Accounting & Finance Jobs',
 'Sales Jobs',
 'Hospitality & Catering Jobs',
 'Teaching Jobs',
 'PR, Advertising & Marketing Jobs']

In [93]:
category2 = df2.Category.value_counts().index.tolist()
category2

['Information Technology',
 'Health',
 'Engineering',
 'Finance',
 'Sales',
 'Hospitality',
 'Education',
 'Marketing']

In [94]:
# Code to fix the conflict
for i in range(len(df2['Category'])):
    ind = category2.index(df2['Category'].iloc[i])
    df2['Category'].iloc[i] = category1[ind]
df2['Category'].value_counts()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11800\198394117.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Category'].iloc[i] = category1[ind]


IT Jobs                             1317
Healthcare & Nursing Jobs            789
Engineering Jobs                     744
Accounting & Finance Jobs            668
Sales Jobs                           498
Hospitality & Catering Jobs          399
Teaching Jobs                        326
PR, Advertising & Marketing Jobs     259
Name: Category, dtype: int64

#### 2.2 Schema conflict: Type column and ContractTime column

In [95]:
df1['ContractTime'].value_counts()

permanent        31019
non-specified    14095
contract          5629
Name: ContractTime, dtype: int64

In [96]:
df2['Type'].value_counts()

Permanent              3040
Fixed Term Contract     540
Name: Type, dtype: int64

In [97]:
def fix_type(val):
    mod = 'non-specified'
    if val == 'Permanent':
        mod =  'permanent'
    elif val == 'Fixed Term Contract':
        mod = 'contract'
    return mod

In [98]:
df2['Type'] = df2['Type'].apply(fix_type)
df2['Type'].value_counts()

permanent        3040
non-specified    1420
contract          540
Name: Type, dtype: int64

In [99]:
df2.rename(columns={'Type': 'ContractTime'}, inplace=True)

#### 2.3 Schema conflict: Monthly Payment column and Salary column

In [100]:
df2['Monthly Payment'] = df2['Monthly Payment'].apply(lambda x: x * 12)

In [101]:
df2.rename(columns={'Monthly Payment': 'Salary'}, inplace=True)

In [102]:
print(df2.columns)
print(df1.columns)


Index(['Salary', 'Closing', 'Organisation', 'Location', 'Category',
       'Full-Time Equivalent (FTE)', 'Opening', 'Job Title', 'ContractTime'],
      dtype='object')
Index(['Id', 'Title', 'Location', 'Company', 'ContractType', 'ContractTime',
       'Category', 'Salary', 'OpenDate', 'CloseDate', 'SourceName'],
      dtype='object')


####  2.4 Schema conflict: OpenDate and CloseDate & Opening and Closing naming conflict

In [103]:
df2.rename(columns={'Opening': 'OpenDate', 'Closing': 'CloseDate'}, inplace=True)

#### 2.5 Schema conflict: Job Title column and Title column naming conflict

In [104]:
df2.rename(columns={'Job Title': 'Title'}, inplace=True)

#### 2.6 Schema conflict: Organization column and Company column naming conflict

In [105]:
df2.rename(columns={'Organisation': 'Company'}, inplace=True)

#### 2.7 Schema conflict: FTE column and ContractType column

In [106]:
df1['ContractType'].value_counts()

non-specified    38078
full_time        11244
part_time         1421
Name: ContractType, dtype: int64

In [107]:
df2.rename(columns={'Full-Time Equivalent (FTE)': 'ContractType'}, inplace=True)
df2['ContractType'].value_counts()

1.0    4834
0.4      47
0.8      41
0.6      39
0.2      39
Name: ContractType, dtype: int64

In [108]:
df2['ContractType'] = df2['ContractType'].apply(lambda x: 'full_time' if int(x) == 1 else ('part_time' if int(x) < 1 else 'non-specified'))
df2['ContractType'].value_counts()

full_time    4834
part_time     166
Name: ContractType, dtype: int64

#### 2.8 Schema conflict: Seconnd dataset does not have Id column

In [109]:
bound = df1.Id.max() + 1
bound

72705204

In [110]:
df2['Id'] = [bound + i for i in range(0, len(df2['Title']))]

In [111]:
df2.Id

0       72705204
1       72705205
2       72705206
3       72705207
4       72705208
          ...   
4995    72710199
4996    72710200
4997    72710201
4998    72710202
4999    72710203
Name: Id, Length: 5000, dtype: int64

#### 2.9 Schema conflict: 2nd dataset does not have SourceName column

In [112]:
df2["SourceName"] = ['non-specified' for i in range((df2.shape[0]))]

#### 2.10 Adapt datatypes across schemas S1 and S2

In [113]:
print (df1.info())
print (df2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50743 entries, 0 to 50742
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            50743 non-null  int64  
 1   Title         50743 non-null  object 
 2   Location      50743 non-null  object 
 3   Company       50743 non-null  object 
 4   ContractType  50743 non-null  object 
 5   ContractTime  50743 non-null  object 
 6   Category      50743 non-null  object 
 7   Salary        50743 non-null  float64
 8   OpenDate      50743 non-null  object 
 9   CloseDate     50743 non-null  object 
 10  SourceName    50743 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 4.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Salary        5000 non-null   float64
 1   CloseDate     5000 non-n

In [114]:
for y in [c for c in df1.columns if c in df2.columns]: # common columns
    if df1[y].dtype != df2[y].dtype:
        print ("Column " + y + "in df1: "+ str(df1[y].dtype) + "to" + str(df2[y].dtype))
        df2[y] = df2[y].astype(df1[y].dtype) 

### 3. Merging data

In [115]:
# Code to merge two data sets
df = pd.concat([df1, df2])
df

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,71846213,"Nurse : Medical/Surgical Roles NHS Trust, SW E...",Devon,non-specified,non-specified,permanent,Healthcare & Nursing Jobs,24400.00,2012-02-03 15:00:00,2012-03-04 15:00:00,jobs4medical.co.uk
1,68580301,Dental : Associate Dentist Jobs in Somerset,Somerset,non-specified,non-specified,non-specified,Healthcare & Nursing Jobs,59000.00,2012-09-26 00:00:00,2012-11-25 00:00:00,jobs4medical.co.uk
2,71337916,Allied Health Care Professional : Optical Advi...,Cheshire,non-specified,non-specified,non-specified,Healthcare & Nursing Jobs,14000.00,2012-11-28 15:00:00,2013-01-27 15:00:00,jobs4medical.co.uk
3,71353660,Dental : Associates Jobs Dentist Job in Darlin...,Durham,non-specified,non-specified,non-specified,Healthcare & Nursing Jobs,28500.00,2013-05-24 00:00:00,2013-06-07 00:00:00,jobs4medical.co.uk
4,71337991,Allied Health Care Professional : Dispensing O...,Essex,non-specified,non-specified,non-specified,Healthcare & Nursing Jobs,22000.00,2013-07-11 00:00:00,2013-08-10 00:00:00,jobs4medical.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
4995,72710199,Head of Regulatory Policy,London,Jonathan Lee Recruitment Ltd,full_time,permanent,Accounting & Finance Jobs,69999.96,2013-01-30 00:00:00,2013-03-31 00:00:00,non-specified
4996,72710200,Android Developer eMarkets and eCommerce Ber...,Berkshire,Impact 360,full_time,permanent,IT Jobs,60000.00,2012-06-28 12:00:00,2012-08-27 12:00:00,non-specified
4997,72710201,SAS Senior Collections Recoveries Analyst,London,Aspire Data Recruitment,full_time,permanent,Accounting & Finance Jobs,33999.96,2013-04-24 00:00:00,2013-07-23 00:00:00,non-specified
4998,72710202,Lead PHP Magento Developer London ****k,London,Annapurna HR Ltd,full_time,permanent,IT Jobs,57500.04,2012-11-18 15:00:00,2012-12-18 15:00:00,non-specified


### 4. Resolving data conflicts:

#### Conflict 1: Index conflict 

In [116]:
df.reset_index(inplace=True)
del df['index']
df

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,71846213,"Nurse : Medical/Surgical Roles NHS Trust, SW E...",Devon,non-specified,non-specified,permanent,Healthcare & Nursing Jobs,24400.00,2012-02-03 15:00:00,2012-03-04 15:00:00,jobs4medical.co.uk
1,68580301,Dental : Associate Dentist Jobs in Somerset,Somerset,non-specified,non-specified,non-specified,Healthcare & Nursing Jobs,59000.00,2012-09-26 00:00:00,2012-11-25 00:00:00,jobs4medical.co.uk
2,71337916,Allied Health Care Professional : Optical Advi...,Cheshire,non-specified,non-specified,non-specified,Healthcare & Nursing Jobs,14000.00,2012-11-28 15:00:00,2013-01-27 15:00:00,jobs4medical.co.uk
3,71353660,Dental : Associates Jobs Dentist Job in Darlin...,Durham,non-specified,non-specified,non-specified,Healthcare & Nursing Jobs,28500.00,2013-05-24 00:00:00,2013-06-07 00:00:00,jobs4medical.co.uk
4,71337991,Allied Health Care Professional : Dispensing O...,Essex,non-specified,non-specified,non-specified,Healthcare & Nursing Jobs,22000.00,2013-07-11 00:00:00,2013-08-10 00:00:00,jobs4medical.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
55738,72710199,Head of Regulatory Policy,London,Jonathan Lee Recruitment Ltd,full_time,permanent,Accounting & Finance Jobs,69999.96,2013-01-30 00:00:00,2013-03-31 00:00:00,non-specified
55739,72710200,Android Developer eMarkets and eCommerce Ber...,Berkshire,Impact 360,full_time,permanent,IT Jobs,60000.00,2012-06-28 12:00:00,2012-08-27 12:00:00,non-specified
55740,72710201,SAS Senior Collections Recoveries Analyst,London,Aspire Data Recruitment,full_time,permanent,Accounting & Finance Jobs,33999.96,2013-04-24 00:00:00,2013-07-23 00:00:00,non-specified
55741,72710202,Lead PHP Magento Developer London ****k,London,Annapurna HR Ltd,full_time,permanent,IT Jobs,57500.04,2012-11-18 15:00:00,2012-12-18 15:00:00,non-specified


#### Finding global key for the data
Since Id is the only column which has no duplications, Id is the global key

In [117]:
# Code to identify global key
print("Number of duplications in each column:")
for col in df.columns:
    print(col + ": " + str(df[col].duplicated().sum()))

Number of duplications in each column:
Id: 0
Title: 303
Location: 55264
Company: 46631
ContractType: 55740
ContractTime: 55740
Category: 55735
Salary: 50925
OpenDate: 53550
CloseDate: 53326
SourceName: 55635


### 5. Saving the integrated and reshaped data
The last part of the integration process is to export our output data to csv format, named as:
- '\<student\_id\>_dataset_integrated.csv'

In [118]:
df.to_csv('./s3927198_dataset_integrated.csv')

## Summary of the Assessment Task
In short, the assessment has demonstrated the different aspects of the 2 datasets and how to integrate them properly. I have learnt a lot from the task and have a great experience.